In [1]:
#!pip install stop-words
#!pip install pyLDAvis
#!pip install xlrd
#!pip install openpyxl
#!pip install --upgrade openpyxl==2.6.0
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import gensim
import pyLDAvis.gensim_models
pattern = r'\b[^\d\W]+\b'
tokenizer = RegexpTokenizer(pattern)
en_stop = get_stop_words('en')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


## **Load Dataset**

In [2]:
cust_feed= pd.read_excel('/content/sample_data/customer_feedback.xlsx', sheet_name='Sheet1', engine='openpyxl')
cust_feed.head()

/usr/local/lib/python3.7/dist-packages/openpyxl/xml/functions.py:30: DeprecationWarning: defusedxml.cElementTree is deprecated, import from defusedxml.ElementTree instead.
  from defusedxml.cElementTree import iterparse
/usr/local/lib/python3.7/dist-packages/openpyxl/xml/functions.py:31: DeprecationWarning: defusedxml.lxml is no longer supported and will be removed in a future release.
  from defusedxml.lxml import fromstring as _fromstring, tostring


,reviews
0,tier_company customer support was very helpful...
1,Too slow. I think I knew more about products ...
2,Well we always buy tier_company for our truck ...
3,The call in person was helpful when I finally ...
4,i WOULD NOT PURCHASE tier_company TIRES IF THE...


**List for Tokenized Documents**

In [3]:
# list for tokenized documents in loop
texts = []

# loop through document list
for i in cust_feed['reviews'].iteritems():
    # clean and tokenize document string
    raw = str(i[1]).lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [raw for raw in tokens if not raw in en_stop]
    
    # lemmatize tokens
    lemma_tokens = [lemmatizer.lemmatize(tokens) for tokens in stopped_tokens]
    
    # remove word containing only single char
    new_lemma_tokens = [raw for raw in lemma_tokens if not len(raw) == 1]
    
    # add tokens to list
    texts.append(new_lemma_tokens)

# sample data
print(texts[0])

['tier_company', 'customer', 'support', 'helpful', 'answering', 'question', 'regarding', 'tire', 'vehicle', 'tier_company', 'website', 'indicated', 'tire', 'looking', 'fit', 'vehicle', 'dealer', 'website', 'indicated', 'tire', 'customer', 'support', 'able', 'answer', 'concern', 'regarding', 'tire', 'fit', 'vehicle', 'impressed', 'service']


In [4]:
texts

[['tier_company',
  'customer',
  'support',
  'helpful',
  'answering',
  'question',
  'regarding',
  'tire',
  'vehicle',
  'tier_company',
  'website',
  'indicated',
  'tire',
  'looking',
  'fit',
  'vehicle',
  'dealer',
  'website',
  'indicated',
  'tire',
  'customer',
  'support',
  'able',
  'answer',
  'concern',
  'regarding',
  'tire',
  'fit',
  'vehicle',
  'impressed',
  'service'],
 ['slow',
  'think',
  'knew',
  'product',
  'chat',
  'service',
  'wanted',
  'help',
  'deciding',
  'two',
  'type',
  'tier_company',
  'tire',
  'help',
  'nothing',
  'find',
  'literature',
  'already',
  'studied',
  'explained',
  'need',
  'seeking',
  'choice',
  'help',
  'help'],
 ['well',
  'always',
  'buy',
  'tier_company',
  'truck',
  'first',
  'time',
  'used',
  'brand',
  'husband',
  'bike',
  'defective',
  'may',
  'recommend',
  'tire',
  'truck',
  'tier_company',
  'bike',
  'end',
  'life',
  'quickly',
  'don',
  'notice',
  'defective',
  'tire',
  'motorc

**Converting Document to terms and document-term matrix**

In [5]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# **Generate LDA Model**
#### **For 20 topics with top 20 words**

In [7]:
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=20, id2word = dictionary, passes=20)
import pprint
pprint.pprint(lda_model.top_topics(corpus,topn=20))

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

[([(0.080915734, 'tire'),
   (0.054341413, 'customer'),
   (0.048464224, 'tier_company'),
   (0.033713706, 'service'),
   (0.012922206, 'warranty'),
   (0.012732098, 'good'),
   (0.0126527855, 'will'),
   (0.011963251, 'vehicle'),
   (0.010186042, 'new'),
   (0.009671255, 'great'),
   (0.009601333, 'always'),
   (0.0095948065, 'set'),
   (0.008893943, 'excellent'),
   (0.008822287, 'product'),
   (0.008809314, 'experience'),
   (0.007961587, 'issue'),
   (0.007896725, 'support'),
   (0.0075049326, 'brand'),
   (0.0070114695, 'car'),
   (0.006738155, 'thank')],
  -2.305624179037925),
 ([(0.061118726, 'tire'),
   (0.05226716, 'tier_company'),
   (0.018083572, 'customer'),
   (0.013663784, 'always'),
   (0.013555358, 'set'),
   (0.011393971, 'two'),
   (0.011193013, 'will'),
   (0.0107081095, 've'),
   (0.01050666, 'brand'),
   (0.009440475, 'great'),
   (0.008692829, 'mile'),
   (0.008508688, 'new'),
   (0.00846628, 'warranty'),
   (0.008257385, 'help'),
   (0.008118078, 'last'),
   (0.0

# **Visualize Topic Model for term frequency**

In [8]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:302: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_table['Freq'] = token_table['Freq'].round()


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.110527  0.053097       1        1  17.597235
13    -0.102011 -0.003348       2        1  16.970936
10    -0.087425  0.004767       3        1   6.597728
4     -0.053341 -0.006627       4        1   6.527452
17    -0.102376 -0.039178       5        1   5.698217
2     -0.055640  0.060647       6        1   5.697125
0     -0.052327  0.014474       7        1   4.916138
14    -0.050715  0.016569       8        1   4.432940
12    -0.048345  0.028552       9        1   4.286071
6      0.000711  0.061136      10        1   3.624108
3     -0.007213 -0.008633      11        1   3.398366
8     -0.037806 -0.078709      12        1   3.155021
7      0.000381  0.027449      13        1   3.028229
5      0.120452  0.118991      14        1   2.822949
16     0.090788 -0.017640      15        1   2.349748
19     0.044566 -0.096618      16        1   2.219611
9      0.079234 -0.102831      17        1   1.818268
1      0.179798  0.038964      18        1   1.798073
11     0.121596  0.049515      19        1   1.700107
15     0.070198 -0.120575      20        1   1.361678, topic_info=             Term        Freq       Total Category  logprob  loglift
15   tier_company  524.000000  524.000000  Default  30.0000  30.0000
16           tire  920.000000  920.000000  Default  29.0000  29.0000
4        customer  345.000000  345.000000  Default  28.0000  28.0000
11       question   64.000000   64.000000  Default  27.0000  27.0000
128          will  116.000000  116.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
387            ve    0.778037   46.524193  Topic20  -5.5370   0.2055
446          last    0.728520   45.028098  Topic20  -5.6027   0.1724
164          hope    0.728480   13.540844  Topic20  -5.6028   1.3739
304          even    0.728469   32.597356  Topic20  -5.6028   0.4954
91           year    0.728463   88.071283  Topic20  -5.6028  -0.4985

[1423 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
333       1  0.179009       abc
333       2  0.358018       abc
333       4  0.089505       abc
333      14  0.089505       abc
333      16  0.268514       abc
...     ...       ...       ...
290      13  0.059388       yet
290      15  0.118775       yet
290      18  0.059388       yet
290      19  0.059388       yet
1561     13  0.615933  yokohama

[3281 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 14, 11, 5, 18, 3, 1, 15, 13, 7, 4, 9, 8, 6, 17, 20, 10, 2, 12, 16])

# **Text Cleansing**




In [9]:
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

# cleaning master function
def clean_tweet(reviews, bigrams=False):
    reviews = reviews.lower() # lower case
    reviews = re.sub('['+my_punctuation + ']+', ' ', reviews) # strip punctuation
    reviews = re.sub('\s+', ' ', reviews) #remove double spacing
    reviews = re.sub('([0-9]+)', '', reviews) # remove numbers
    reviews_token_list = [word for word in reviews.split(' ')
                            if word not in my_stopwords] # remove stopwords

    #reviews_token_list = [word_rooter(word) if '#' not in word else word
                        #for word in reviews_token_list] # apply word rooter
    if bigrams:
        reviews_token_list = reviews_token_list+[reviews_token_list[i]+'_'+reviews_token_list[i+1]
                                            for i in range(len(reviews_token_list)-1)]
    reviews = ' '.join(reviews_token_list)
    return reviews

<input>:9: DeprecationWarning: invalid escape sequence \s
<input>:9: DeprecationWarning: invalid escape sequence \s
<input>:9: DeprecationWarning: invalid escape sequence \s
<input>:9: DeprecationWarning: invalid escape sequence \s
<input>:9: DeprecationWarning: invalid escape sequence \s
<input>:9: DeprecationWarning: invalid escape sequence \s
<input>:9: DeprecationWarning: invalid escape sequence \s
<input>:9: DeprecationWarning: invalid escape sequence \s
<input>:9: DeprecationWarning: invalid escape sequence \s
<ipython-input-9-15f27a809fdc>:9: DeprecationWarning: invalid escape sequence \s
  reviews = re.sub('\s+', ' ', reviews) #remove double spacing


In [10]:
cust_feed['cleaned_reviews'] = cust_feed.reviews.apply(clean_tweet)
cust_feed.head()

,reviews,cleaned_reviews
0,tier_company customer support was very helpful...,tier company customer support helpful answerin...
1,Too slow. I think I knew more about products ...,slow think knew products chat service wanted h...
2,Well we always buy tier_company for our truck ...,well always buy tier company truck first time ...
3,The call in person was helpful when I finally ...,call person helpful finally able talk someone ...
4,i WOULD NOT PURCHASE tier_company TIRES IF THE...,would purchase tier company tires car requires...


# **Feature Extraction**

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# the vectorizer object will be used to transform text to vector form
vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+')

# apply transformation
tf = vectorizer.fit_transform(cust_feed['cleaned_reviews']).toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()

<input>:4: DeprecationWarning: invalid escape sequence \w
<input>:4: DeprecationWarning: invalid escape sequence \w
<input>:4: DeprecationWarning: invalid escape sequence \w
<input>:4: DeprecationWarning: invalid escape sequence \w
<input>:4: DeprecationWarning: invalid escape sequence \w
<input>:4: DeprecationWarning: invalid escape sequence \w
<input>:4: DeprecationWarning: invalid escape sequence \w
<ipython-input-11-aaeb7101dfdf>:4: DeprecationWarning: invalid escape sequence \w
  vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+')


In [12]:
from sklearn.decomposition import LatentDirichletAllocation
number_of_topics = 20
model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

In [13]:
model.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [14]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        #topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        #for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [16]:
no_top_words = 20
display_topics(model, tf_feature_names, no_top_words)

,Topic 0 words,Topic 1 words,Topic 2 words,Topic 3 words,Topic 4 words,Topic 5 words,Topic 6 words,Topic 7 words,Topic 8 words,Topic 9 words,Topic 10 words,Topic 11 words,Topic 12 words,Topic 13 words,Topic 14 words,Topic 15 words,Topic 16 words,Topic 17 words,Topic 18 words,Topic 19 words
0,products,customer,company,tier,issue,customer,tires,first,get,years,warranty,tires,tires,tire,help,purchase,customer,tires,customer,tier
1,company,service,tier,company,response,service,tier,tier,time,tires,tires,happy,new,company,question,helpful,tires,like,tires,tires
2,tier,company,tires,always,resolved,support,company,received,experience,tier,still,wear,miles,tier,answer,go,one,good,back,company
3,recommend,tier,purchased,tires,claim,quickly,last,company,long,company,tire,ride,car,dealer,would,vehicle,supportlaced,vehicle,problem,since
4,customer,thank,brand,great,warranty,questions,sets,set,took,many,even,new,set,buy,received,would,best,tier,got,told
5,discount,supportresentative,wear,quality,tire,thanks,vehicles,pleased,good,used,mileage,disappointed,really,product,never,company,supportlace,company,could,however
6,would,excellent,great,customer,tread,supportlacement,performance,tires,tires,cars,get,much,tier,tires,got,tires,company,work,satisfied,buying
7,service,good,purchase,best,thank,warranty,always,well,back,always,go,bought,brand,problem,told,tier,tier,also,never,bought
8,even,helpful,satisfied,service,still,helpful,good,would,product,continue,dealer,set,bought,recommend,get,car,miles,would,care,one
9,quality,received,set,set,wear,excellent,mileage,still,dealer,great,thank,mileage,buy,need,tires,never,even,thank,service,new
